In [3]:
import openai
import os

In [2]:
from openai import OpenAI

In [6]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

In [9]:
client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': 'user', 'content': 'Is OpenAI free to use?'}]
)

ChatCompletion(id='chatcmpl-BfoeBpdkWw5wnEIzpd72QCtVS4yYX', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='OpenAI offers both free and paid options for accessing its models and tools. For instance:\n\n1. **ChatGPT**:\n   - There is a free tier available. Anyone can sign up and use it, though there might be limitations on usage and access compared to the paid version.\n   - OpenAI also offers a subscription plan called "ChatGPT Plus" for a monthly fee. This provides benefits such as faster response times, priority access during peak times, and access to newer models.\n\n2. **API Access**:\n   - OpenAI\'s API has a pricing model based on usage. Developers and businesses can sign up and use the API, but beyond initial credits, usage incurs costs based on the amount of processing and data involved.\n\n3. **Research and Education**:\n   - Occasionally, OpenAI offers grants or credits for research and educational purposes, allowing certai